In [1]:
import ast
import pandas as pd
import pickle
import gensim
import math

In [2]:
name_list = []
with open('demoFiles/testData') as f:
    d = f.readlines()
    for i in d:
        name_list.append(ast.literal_eval(i.split("|")[1])['name'])

In [3]:
patientName = "Kasey Veum"

In [4]:
def testPatientDetails(line):
        dic = line.split('|')[1]
        dic = ast.literal_eval(dic)
        name = dic['name']
        if dic['gender'] == 'M':
            gender = 'Male'
        else:
            gender = 'Female'
        age = int(round(dic['age'], 0))
        race = dic['race']
        zip_code = dic['zip']
        state = dic['state']

        patientDetail = dict()
        patientDetail['Name'] = name
        patientDetail['Age'] = str(age)
        patientDetail['Gender'] = gender
        patientDetail['Race'] = race
        patientDetail['Zip'] = str(zip_code)
        patientDetail['State'] = state
        return patientDetail

In [5]:
def similaritiesPredictDuplicate(patientName):
        filename = patientName.replace(" ", "_")+'.csv'
        demo_df = pd.read_csv('./demoFiles/'+filename)
        demo_df = demo_df.head()
        demo_list = demo_df.values.tolist()
        headers = demo_df.columns.tolist()
        demo_list.insert(0,headers)

        '''with open('demoFiles/'+filename, encoding="utf8") as f:
            lines = f.readlines()
            demo_list = []
            for line in lines:
                l = line.split(",")
                l[-1] = l[-1].replace('\n','')
                demo_list.append(l)'''
        return demo_list

In [6]:
with open('demoFiles/testData') as f:
    lines = f.readlines()
    index = [i for i, e in enumerate(lines) if (patientName in e)]
    line = lines[index[0]].split('\n')[0]
    feed_events = line.split("|")[2].split(" ")
patientDetail = testPatientDetails(line)
print(patientDetail)
print(type(patientDetail))
'''#Original Code
predictions, similarities =retrievePatientDetails.predict(feed_events)        
patient_json = retrievePatientDetails.similaritiesPredict(similarities,patientName)
'''
#Code for improve Performance
patient_json = similaritiesPredictDuplicate(patientName)

{'Name': 'Kasey Veum', 'Age': '84', 'Gender': 'Female', 'Race': 'white', 'Zip': '46151', 'State': 'Indiana'}
<class 'dict'>


In [7]:
def similaritiesPredictDuplicateFilter(patientName,lists,patientDetail):
        filename = patientName.replace(" ", "_")+'.csv'
        demo_df = pd.read_csv('./demoFiles/'+filename)
        for i in lists[1:]:
            if i == 'Age' or i =='Zip':
                value = int(patientDetail.get(i))
            else:
                value = patientDetail.get(i)
            demo_df = demo_df[demo_df[i]==value]
        demo_df = demo_df.head()
        if (demo_df.shape[0] != 0):
            demo_list = demo_df.values.tolist()
            headers = demo_df.columns.tolist()
            demo_list.insert(0,headers)
            return demo_list
        else:
            return 'Nothing'

In [8]:
lists = ["Kasey Veum","Gender"]
patientName = lists[0]
with open('demoFiles/testData') as f:
    lines = f.readlines()
    index = [i for i, e in enumerate(lines) if (patientName in e)]
    line = lines[index[0]].split('\n')[0]
    feed_events = line.split("|")[2].split(" ")
patientDetail = testPatientDetails(line)
#Code for improve Performance
patient_json = similaritiesPredictDuplicateFilter(patientName,lists,patientDetail)
print(patient_json)

[['Name', 'Age', 'Gender', 'Race', 'Zip', 'State', 'Similarity'], ['Rana Sanford', 89, 'Female', 'white', 46936, 'Indiana', 99.84200000000001], ['Marva Macejkovic', 85, 'Female', 'white', 46151, 'Indiana', 99.76799999999999], ['Edie Turcotte', 85, 'Female', 'white', 46163, 'Indiana', 99.71], ['Timothy Bernier', 85, 'Female', 'white', 47346, 'Indiana', 99.697], ['Joella Herzog', 85, 'Female', 'white', 46580, 'Indiana', 99.65799999999999]]


In [9]:
#Dummy Code to increase response time of Performance
patientNameList = []
with open('uploaded_files/test_user') as f:
    lines = f.readlines()
    for line in lines:
        patientNameList = patientNameList + [line.split("|")[1].split(",")[1].split(":")[1].split('"')[1]]
    
print(patientNameList)

['Noel Balistreri', 'Sunny Marquardt', 'Dannie Hand', 'Russel Hessel', 'Hugo Saldivar', 'Mardell Fay']


In [10]:
def predict(feed_events):
    vocab = pickle.load(open('uploaded_files/vocab_user.pkl', 'rb'))
    size = pickle.load(open('uploaded_files/size_user.pkl', 'rb'))
    decay = pickle.load(open('uploaded_files/decay_user.pkl', 'rb'))
    model = pickle.load(open('uploaded_files/model_user.pkl', 'rb'))
    diags = pickle.load(open('uploaded_files/diags_user.pkl', 'rb'))
    pat_diag = pickle.load(open('uploaded_files/pat_diag_user.pkl', 'rb'))
    pat_vec = pickle.load(open('uploaded_files/pat_vec_user.pkl', 'rb'))

    vec = [0] * size
    te = len(feed_events) * 1.0
    weighted_events = [(e, math.exp(decay * (i - te + 1) / te))
                       for i, e in enumerate(feed_events) if e in vocab]

    for e, d in weighted_events:
        vec = [x * d + y for x, y in zip(model[e].tolist(), vec)]

    mag = math.sqrt(sum([x ** 2 for x in vec]))
    sim = []
    for e in pat_vec:
        sim.append(sum([(x * y) for x, y in zip(e, vec)]) /
                   (mag * math.sqrt(sum([x ** 2 for x in e]))))

    avg_sim = sum(sim) / len(sim)
    predictions = {}
    for d in diags:
        probability = 0
        norm = 0

        for i, pat_sim in enumerate(sim):
            if (d in pat_diag[i]) & (pat_sim >= .90):
                probability += max(0, pat_sim - avg_sim)
            norm += max(0, pat_sim - avg_sim)

        predictions[d] = probability * 1.0 / norm
    return predictions,sim

def testPatientDetails(line):
    dic = line.split('|')[1]
    dic = ast.literal_eval(dic)
    '''name = dic['name']
    if dic['gender'] == 'M':
        gender = 'Male'
    else:
        gender = 'Female'
    age = int(round(dic['age'], 0))
    race = dic['race']
    zip_code = dic['zip']
    state = dic['state']

    patientDetail = dict()
    patientDetail['Name'] = name
    patientDetail['Age'] = str(age)
    patientDetail['Gender'] = gender
    patientDetail['Race'] = race
    patientDetail['Zip'] = str(zip_code)
    patientDetail['State'] = state'''
    return dic
def similaritiesPredict(similarities):
    sim_df = pd.DataFrame(similarities).reset_index()
    top_sim = sim_df.sort_values(by=0, ascending=False)[0].tolist()
    value = sim_df.sort_values(by=0, ascending=False).index.tolist()
    iterator = 0
    with open('uploaded_files/train_user', encoding="utf8") as f:
        line = f.readlines()
        list_dic = []
        for v in value:
            dic = line[v].split('|')[1]
            dic = ast.literal_eval(dic)
            dic['Similarity'] = round((top_sim[iterator] * 100), 3)
            if(dic['Similarity']<100):
                list_dic.append(dic)
            iterator = iterator + 1
            col_list = list(dic.keys())
        demo_df = pd.DataFrame(list_dic)
        demo_df = demo_df[col_list]
    headers = demo_df.columns.tolist()
    data = demo_df.values.tolist()
    data.insert(0, headers)
    return data

In [11]:
patientName = "Sunny Marquardt"
with open('uploaded_files/test_user') as f:
    lines = f.readlines()
    index = [i for i, e in enumerate(lines) if (patientName in e)]
    line = lines[index[0]].split('\n')[0]
    feed_events = line.split("|")[2].split(" ")
predictions, similarities =predict(feed_events)
patientDetail = testPatientDetails(line)
patient_json = similaritiesPredict(similarities)
print(patient_json)

[['id', 'name', 'race', 'gender', 'zip', 'state', 'socioeconomic_status', 'current_age', 'Similarity'], ['983f67d0-3859-4431-90ae-a07349af8f0f', 'Russel Hessel', 'white', 'M', '46706', 'Indiana', 'Low', '85', 99.999], ['474f102b-afc2-47e5-94a6-cfa014090a14', 'Walker Kub', 'white', 'M', '46360', 'Indiana', 'High', '85', 99.999], ['781112bd-bc59-465a-aa98-80622bb7e5da', 'Noel Balistreri', 'black', 'M', '47708', 'Indiana', 'Middle', '84', 99.999], ['3ca533b4-a5f7-447e-8deb-ab41ba0f86a6', 'Elisa Metz', 'white', 'F', '46347', 'Indiana', 'Middle', '86', 99.998], ['1a1eacb9-ed3c-4bc1-80be-d4f5487cc037', 'Jamie Mann', 'white', 'M', '47043', 'Indiana', 'Middle', '93', 99.998], ['e7d75073-4696-4fe9-a1d2-ae3fab4207ae', 'Mardell Fay', 'white', 'F', '46346', 'Indiana', 'Middle', '91', 99.998], ['5593c800-ae80-4ec7-ae6f-a022e0edb521', 'Hana Bashirian', 'white', 'F', '47971', 'Indiana', 'Middle', '98', 99.997], ['77927499-42d3-4c0c-b80f-b988ee84a187', 'Dee Bernhard', 'white', 'M', '46216', 'Indiana',

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
